In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
movies = pd.read_csv(Path("archive/movies.csv"))


In [140]:
import re
re.findall(r'\d+', "Diplomatic Immunity 4375 (2009- )")

['4375', '2009']

In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
title_arr = movies['title'].to_numpy()
title_arr[0].split(')')

['Toy Story (1995', '']

In [13]:
movi_name = []
year_rel = []
for str_ in title_arr:
    str_arr = str_.split(')')
    title = ''.join(str_arr[:-1])

    if (title[-4:].isdigit()):
        movi_name.append(title[:-5])
        year_rel.append(title[-4:])
    else:
        movi_name.append(str_)
        year_rel.append('NaN')
len(movi_name),len(year_rel)

(58098, 58098)

In [14]:
movies['movie_name'] = movi_name
movies['year_released'] = year_rel

In [16]:
movies.drop(['title'], axis=1, inplace= True)

In [18]:
movies[movies['movieId']==192339]

,movieId,genres,movie_name,year_released
57504,192339,(no genres listed),The Sign of Three,NaN


In [19]:
no_year_movies = movies[movies['year_released']=='NaN']
no_year_movies

,movieId,genres,movie_name,year_released
10023,32930,Action|Drama,Category 6: Day of Destruction,NaN
10613,40697,Sci-Fi,Babylon 5,NaN
15719,79607,Action|Drama|Sci-Fi|Thriller,"Millions Game, The (Das Millionenspiel)",NaN
17444,87442,Documentary,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)",NaN
22651,107434,Comedy,Diplomatic Immunity (2009– ),NaN
...,...,...,...,...
57504,192339,(no genres listed),The Sign of Three,NaN
57674,192829,Adventure|Children|Romance,Parwaaz Hai Junoon,NaN
57789,193149,(no genres listed),¿Qué te juegas?,NaN
57903,193443,(no genres listed),Wall,NaN


In [20]:
tags = pd.read_csv(Path("archive/tags.csv"))
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [21]:
tag_movId = tags['movieId'].values 
mov_Id = movies['movieId'].values
len(list(set(mov_Id).difference(set(tag_movId))))

12117

In [22]:
arr = movies[movies["genres"]== '(no genres listed)']['movieId'].values
movies[movies["genres"]== '(no genres listed)']

,movieId,genres,movie_name,year_released
16662,83773,(no genres listed),Away with Words (San tiao ren,1999
16677,83829,(no genres listed),Scorpio Rising,1964
16856,84768,(no genres listed),Glitterbug,1994
17178,86493,(no genres listed),"Age of the Earth, The (A Idade da Terra",1980
17343,87061,(no genres listed),Trails (Veredas,1978
...,...,...,...,...
58064,193815,(no genres listed),Tempting Fate,1998
58079,193849,(no genres listed),Rafinha Bastos: Ultimatum,2018
58082,193855,(no genres listed),"Spring Night, Summer Night",1967
58090,193870,(no genres listed),Isabelle Eberhardt,1991


In [ ]:
from statistics import mode

no_genre = pd.DataFrame(columns=['movieId', 'genres'])
j=0
for movId in arr:
    tags_m = tags[tags['movieId']==movId]['tag'].values
    reqd_movId= np.array([])
    for tag_val in tags_m:
        reqd_movId = np.append(reqd_movId, tags[tags['tag']== tag_val]['movieId'].values)

    gen_values = np.array([])
    for r_movId in np.unique(reqd_movId):
        gen_values = np.append(gen_values, movies[movies['movieId']== r_movId]['genres'].values)
    #print(gen_values)
    gen_values= np.delete(gen_values, np.where(gen_values == '(no genres listed)'))
    most_common = '(no genres listed)'
    if len(gen_values)!=0:
        most_common = mode(gen_values)
            
    no_genre.loc[j] = [ movId , most_common]
    j=j+1
    #print (movId , most_common)

In [ ]:
no_genre.to_csv(Path('preproc_datasets/no_genre.csv'))

In [23]:
#create a new dataFrame movies_new
#movies_new = pd.DataFrame(columns = movies.columns )
movie_genreID = pd.DataFrame(columns=['movieId', 'genre'])
# movies_new.columns = movies.columns
genre = pd.DataFrame(columns= ['genreId', 'genre'])
j=0
dict={}
gen_id=0
for i in range (len(movies)):
    if (movies.loc[i]['genres'] != '(no genres listed)'):
        A = movies.loc[i]['genres'].split('|')
        #movies_new.loc[i] = [movies.loc[i]['movieId'], movies.loc[i]['movie_name'], movies.loc[i]['year_released']]
        for gen in A:
            if dict.get(gen)== None:
                dict[gen]= gen_id
                genre.loc[gen_id] = [gen_id, gen]
                gen_id= gen_id+1


            movie_genreID.loc[j]= [movies.loc[i]['movieId'], dict[gen]]
            j=j+1
    else:
        movie_genreID.loc[j] = [movies.loc[i]['movieId'], 'NaN']
        j=j+1

#movies_new

In [24]:
genre.loc[19]=[19, 'NaN']
genre

,genreId,genre
0,0,Adventure
1,1,Animation
2,2,Children
3,3,Comedy
4,4,Fantasy
5,5,Romance
6,6,Drama
7,7,Action
8,8,Crime
9,9,Thriller


In [25]:
# for i in range(len(movie_genreID)):
#     if (movie_genreID.iloc[i]['genre'] == 'NaN'):
#         movie_genreID.iloc[i]['genre'] = 19
movie_genreID['genre'].replace(['NaN'], [19], inplace=True)

In [26]:
movie_genreID

,movieId,genre
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
106102,193882,10
106103,193882,12
106104,193886,7
106105,193886,8


In [27]:
movie_genreID.rename(columns = {'genre':'genreId'}, inplace=True)

In [28]:
movie_genreID

,movieId,genreId
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
106102,193882,10
106103,193882,12
106104,193886,7
106105,193886,8


In [29]:
movies

,movieId,genres,movie_name,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
58093,193876,(no genres listed),The Great Glinka,1946
58094,193878,Comedy,Les tribulations d'une caissière,2011
58095,193880,Drama,Her Name Was Mumu,2016
58096,193882,Adventure|Drama|Horror|Sci-Fi,Flora,2017


In [30]:
movies.drop(['genres'], axis=1, inplace=True)

In [34]:
tags['tag'].value_counts()

sci-fi                 9400
atmospheric            6430
action                 6219
comedy                 5923
surreal                5299
                       ... 
Thirlling Climax          1
netwatch                  1
hateuscuztheyaintus       1
addium                    1
rachel mcadams            1
Name: tag, Length: 74714, dtype: int64

In [35]:
no_genre = pd.read_csv(Path('preproc_datasets/no_genre.csv'))

In [40]:
no_genre.drop(['Unnamed: 0'], axis=1, inplace=True)

,movieId,genres
0,83773,Drama
1,83829,Drama
2,84768,Drama
3,86493,Drama
4,87061,Drama
...,...,...
4261,193815,(no genres listed)
4262,193849,(no genres listed)
4263,193855,(no genres listed)
4264,193870,(no genres listed)


In [41]:
no_genre[no_genre['movieId']== 83773]['genres'].values[0]

'Drama'

In [42]:
genre[genre['genre']== 'Drama']['genreId'].values[0]

6

In [43]:
genre.loc[8]['genre']

'Crime'

In [51]:
movie_genreID['genreId'][6]

2

In [52]:
for i in range (len(movie_genreID)):
    gid = movie_genreID.iloc[i]['genreId']
    mid = movie_genreID.iloc[i]['movieId']
    if (gid == 19):
        val = no_genre[no_genre['movieId']== mid]['genres'].values[0]
        if (val != '(no genres listed)'):
            movie_genreID['genreId'][i] = genre[genre['genre']== val]['genreId'].values[0]

In [53]:
len(movie_genreID[movie_genreID['genreId']==19])

2256

In [56]:
genre.to_csv(Path('preproc_datasets/genres.csv'))

In [58]:
movie_genreID.to_csv(Path('preproc_datasets/movie_genres.csv'))

In [60]:
movies.to_csv(Path('preproc_datasets/movies.csv'))

In [ ]:
######################################################### G ###########################################
############################ B #################### N ####################################################
###################### E ##################### I #########################################################
################ W ##############  R R A P ######################################################################
############################ C ########################################################################
######################## s ############################################################################